# EDS 220 Fall 2021

## Visualizing global surface temperature and precipitation using Google Earth Engine

In [55]:
import subprocess

try:
    import geemap
except ImportError:
    print('Installing geemap ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])


Import the Google Earth Engine. Note: this will fail unless you activate the "ee" environment using conda before launching JupyterLab.

In [56]:
import ee
import geemap
import pandas as pd

## Create an interactive map

Now let's load the package that will allow us to map a given dataset. The default basemap is, unsurprisingly, Google Maps.

In [31]:
Map = geemap.Map(center=[40,-100], zoom=2)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

If you execute the above, you can see that the map is created - but there is no data plotted. Now we need to tell GEE which dataset to layer on top of the base map. 

The next chunk of code will load some data; when executed, it will display within the map window above.

## Load data and plot over the basemap

The dataset I've chosen here is the ERA5 "reanalysis": essentially, a reanalysis is a numerical model (in this case, a weather model) that's been forced to match observations of the real world (in this case, atmospheric weather observations) as closely as possible. ERA5 is produced by the European Centre for Medium-Range Weather Forecasts, or ECMWF; so you'll see below that the parent directory for this dataset is called ECMWF. A description of the dataset is here:
https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_DAILY

The below chunk of code loads in the "mean_2m_air_temperature" field, which is basically what it sounds like: the temperature of air at an altitude of 2 meters, everywhere on Earth. Note that the units of temperature are in Kelvin!

In [49]:
# Add Earth Engine dataset

gdat = ee.ImageCollection('ECMWF/ERA5/DAILY')
ts = gdat.select('mean_2m_air_temperature')

pr=gdat.select('total_precipitation')

This data isn't very mappable right now, since it contains information every day from 1979 to 2020. We need to filter it down to a single field for a time of our choice. Let's select an arbitrary time range: here, July of 2019.

In [54]:
tsdflt=ts.filter(ee.Filter.date('2019-07-01', '2019-07-31'));
prdflt=pr.filter(ee.Filter.date('2019-07-01', '2019-07-31'));

# Uncomment one of the following statements to center the map on
# a particular location.
# Map.setCenter(-90.162, 29.8597, 4)   # New Orleans, USA
# Map.setCenter(-114.9774, 31.9254, 4) # Mouth of the Colorado River, Mexico
# Map.setCenter(-111.1871, 37.0963, 3) # Lake Powell, USA
# Map.setCenter(149.412, -35.0789, 4)  # Lake George, Australia
Map.setCenter(105.26, 11.2134, 4)     # Mekong River Basin, SouthEast Asia
# Map.setCenter(90.6743, 22.7382, 4)   # Meghna River, Bangladesh
# Map.setCenter(81.2714, 16.5079, 4)   # Godavari River Basin Irrigation Project, India
# Map.setCenter(14.7035, 52.0985, 4)   # River Oder, Germany & Poland
# Map.setCenter(-59.1696, -33.8111, 4)  # Buenos Aires, Argentina\
#Map.setCenter(-74.4557, -8.4289, 4)  # Ucayali River, Peru


Now we can set a color palette to use when plotting the data layer.

In [51]:
VIS_TS = {
    'min':273,
    'max':310,
    'palette': ['#000080', '#0000D9', '#4000FF', '#8000FF', '#0080FF', '#00FFFF', '#00FF80',
    '#80FF00', '#DAFF00', '#FFFF00', '#FFF500', '#FFDA00', '#FFB000', '#FFA400',
    '#FF4F00', '#FF2500', '#FF0A00', '#FF00FF']
}

VIS_PREC = {
    'min':0,
    'max':0.1,
    'palette': ['#FFFFFF', '#00FFFF', '#0080FF', '#DA00FF', '#FFA400', '#FF0000']
}

In [52]:
Map.addLayer(prdflt, VIS_PREC,'surface air temperature',opacity=0.75)